# Importing the Packages

In [2]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import preprocessing

# Loading the DataSet

In [4]:
titanic_train = pd.read_csv("train.csv")
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
titanic_train["Age"].mean()

32.69985376827896

In [16]:
new_age_var = np.where(titanic_train["Age"].isnull(),32,titanic_train["Age"])
titanic_train["Age"]=new_age_var

# Converting the String to Int

In [17]:
label_encoder = preprocessing.LabelEncoder()

In [20]:
titanic_train["Sex"] = label_encoder.fit_transform(titanic_train["Sex"])
titanic_train["Embarked"] = label_encoder.fit_transform(titanic_train["Embarked"])
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2


# Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
titanic_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [40]:
rf_model = RandomForestClassifier(n_estimators=1000 , max_features=2 , oob_score=True)

In [30]:
features = ['Sex','Embarked','Pclass','Age','SibSp','Parch','Fare']

In [43]:
rf_model.fit(X = titanic_train[features], y=titanic_train["Survived"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [44]:
print("OOB Accuracy")
print(rf_model.oob_score_)

OOB Accuracy
0.8053993250843644


In [46]:
for feature, imp in zip(features,rf_model.feature_importances_):
    print(feature,imp);

Sex 0.26245378577160317
Embarked 0.03496047403686541
Pclass 0.08677650178931261
Age 0.25718388296422906
SibSp 0.049867660896222896
Parch 0.039000132009827215
Fare 0.2697575625319397


# Inference : Important features we get are Sex,Age,Fare

# Decision Tree

In [63]:
tree_model = tree.DecisionTreeClassifier(max_depth = 8 )

In [64]:
predictors = pd.DataFrame([titanic_train["Sex"],titanic_train["Age"],titanic_train["Fare"]]).T

In [66]:
tree_model.fit(X= predictors, y = titanic_train["Survived"])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [67]:
with open("Dtree1.dot","w") as f:
    f= tree.export_graphviz(tree_model,feature_names=["Sex","Age","Fare"],out_file=f)

In [68]:
tree_model.score(X = predictors , y = titanic_train["Survived"])

0.8706411698537683

# PREDICTION

In [69]:
titanic_test = pd.read_csv("test.csv")
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [70]:
titanic_test["Age"].isnull().sum()

0

In [74]:
titanic_test["Sex"]= label_encoder.fit_transform(titanic_test["Sex"])
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,S


In [77]:
test_features = pd.DataFrame([titanic_test["Age"],titanic_test["Sex"],titanic_test["Fare"]]).T

In [80]:
test_preds = tree_model.predict(X = test_features)

In [81]:
predicted_output = pd.DataFrame({"PassengerId":titanic_test["PassengerId"],"Survived":test_preds})

In [85]:
predicted_output.to_csv("Output.csv",index=False)

In [88]:
predicted_output.head()

,PassengerId,Survived
0,892,1
1,893,1
2,894,1
3,895,1
4,896,1
